In [1]:
import sys
sys.path.append("..")
import torch
import torch.onnx
from efficientunet import *
from efficientunet.layers import BatchNorm2d

In [2]:
model = get_efficientunet_b2(out_channels=3, concat_input=True, pretrained=False, bn = BatchNorm2d)
# model.load_state_dict(torch.load('../models/masked/exp6-re/best_5761.pth',map_location = lambaada storage, loc: storage))
model.eval()

EfficientUnet(
  (encoder): Encoder(
    (stem_conv): Conv2dSamePadding(
      2, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 1, 1), value=0.0)
    )
    (stem_batch_norm): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (stem_swish): Swish()
    (blocks): ModuleList(
      (0): MBConvBlock(
        (swish): Swish()
        (_depthwise_conv): Conv2dSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding)

In [3]:
dummy_input = torch.randn((1, 2, 257, 64),requires_grad=False)

In [5]:
%%time
with torch.no_grad():
    torch_out = model(dummy_input)

36.13267183303833 4.76837158203125e-06 0.09547758102416992 6.723403930664062e-05 0.030711889266967773 0.4519362449645996
CPU times: user 3min 54s, sys: 437 ms, total: 3min 54s
Wall time: 43.3 s


In [4]:
model_name = "effb0.onnx"
# Export the model
torch.onnx.export(model,               # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  model_name,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  verbose = True,
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size', 3 : 'n_frame'},    # variable lenght axes
                                'output' : {0 : 'batch_size', 3 : 'n_frame'}}
                 )

../efficientunet/efficientunet.py:76: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if shape_list not in shapes:
../efficientunet/layers.py:204: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assignment). This might cause the trace to be incorrect, because all other views that also reference this data will not reflect this change in the trace! On the other hand, if all other views use the same memory chunk, but are disjoint (e.g. are outputs of torch.split), this might still be safe.
  x_new[:,:,::2,::2] = x.clone()
../efficientunet/layers.py:205: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assi

graph(%input : Float(1, 2, 257, 64),
      %encoder.stem_conv.weight : Float(32, 2, 3, 3),
      %encoder.stem_batch_norm.weight : Float(32),
      %encoder.stem_batch_norm.bias : Float(32),
      %encoder.stem_batch_norm.running_mean : Float(32),
      %encoder.stem_batch_norm.running_var : Float(32),
      %encoder.stem_batch_norm.num_batches_tracked : Long(),
      %encoder.blocks.0._depthwise_conv.weight : Float(32, 1, 3, 3),
      %encoder.blocks.0._bn1.weight : Float(32),
      %encoder.blocks.0._bn1.bias : Float(32),
      %encoder.blocks.0._bn1.running_mean : Float(32),
      %encoder.blocks.0._bn1.running_var : Float(32),
      %encoder.blocks.0._bn1.num_batches_tracked : Long(),
      %encoder.blocks.0._se_reduce.weight : Float(8, 32, 1, 1),
      %encoder.blocks.0._se_reduce.bias : Float(8),
      %encoder.blocks.0._se_expand.weight : Float(32, 8, 1, 1),
      %encoder.blocks.0._se_expand.bias : Float(32),
      %encoder.blocks.0._project_conv.weight : Float(16, 32, 1, 1),
 

In [7]:
import onnx
import onnxruntime as rt

onnx_model = onnx.load('test_gpu.onnx')
onnx.checker.check_model(onnx_model)

In [10]:
import numpy as np